In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model.utils import Site as site

from rectified_logistic import RectifiedLogistic

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)


In [2]:
src = "/home/vishu/repos/hbmep-paper/archived/reports/paper/rats/J_RCML_000/link-comparison/rectified-logistic/Biceps/inference.pkl"
with open(src, "rb") as g:
    model, mcmc, posterior_samples = pickle.load(g)

src = "/home/vishu/data/hbmep-processed/J_RCML_000/data.csv"
df = pd.read_csv(src)

model.toml_path = '/home/vishu/repos/hbmep-paper/archived/configs/paper/rats/J_RCML_000/config.toml'
df, encoder_dict = model.load(df=df)


2024-01-03 15:25:19,680 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/rats/J_RCML_000/link-comparison/rectified-logistic/Biceps
2024-01-03 15:25:19,685 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/rats/J_RCML_000/link-comparison/rectified-logistic/Biceps
2024-01-03 15:25:19,694 - hbmep.dataset.core - INFO - Total non-positive observations: 0
2024-01-03 15:25:19,701 - hbmep.dataset.core - WARNING - Total missing observations: 0
2024-01-03 15:25:19,705 - hbmep.dataset.core - INFO - Processing data ...
2024-01-03 15:25:19,711 - hbmep.utils.utils - INFO - func:load took: 0.03 sec


In [3]:
posterior_samples.keys()

dict_keys(['H', 'H_raw', 'H_scale', 'H_scale_global_scale', 'H_scale_raw', 'L', 'L_raw', 'L_scale', 'L_scale_global_scale', 'L_scale_raw', 'a', 'a_mean', 'a_scale', 'b', 'b_raw', 'b_scale', 'b_scale_global_scale', 'b_scale_raw', 'ell_raw', 'ell_scale', 'ell_scale_global_scale', 'ell_scale_raw', 'g_1', 'g_1_raw', 'g_1_scale', 'g_1_scale_global_scale', 'g_1_scale_raw', 'g_2', 'g_2_raw', 'g_2_scale', 'g_2_scale_global_scale', 'g_2_scale_raw', 'v', 'v_raw', 'v_scale', 'v_scale_global_scale', 'v_scale_raw', 'µ', 'β', 'ℓ'])

In [5]:
combinations = model._make_combinations(df=df, columns=model.features)

In [ ]:
b = posterior_samples["b"]
v = posterior_samples["v"]

In [6]:
prediction_df = model.make_prediction_dataset(df=df, num=1000)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)


2024-01-03 15:30:06,947 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.03 sec


AttributeError: type object 'Site' has no attribute 'g_1'

In [ ]:
a, b, v, L, ell, H = \
    posterior_samples[site.a], \
    posterior_samples[site.b], \
    posterior_samples[site.v], \
    posterior_samples[site.L], \
    posterior_samples[site.ell], \
    posterior_samples[site.H]

features, intensity,= model._collect_regressor(df=prediction_df)
features, n_features = features
intensity, n_data = intensity
intensity = intensity.reshape(-1, 1)
intensity = np.tile(intensity, (1, model.n_response))

feature0 = features[0].reshape(-1,)
feature1 = features[1].reshape(-1,)

params = [a, b, v, L, ell, H]
params = [u[:, feature0, feature1] for u in params]

fn_out = F.rectified_logistic(intensity, *tuple(params))
fn_out = np.array(fn_out)

fn_prime_out = F.prime(F.rectified_logistic, np.tile(intensity, (params[0].shape[0], 1, 1)), *tuple(params))
fn_prime_out = np.array(fn_prime_out) * time_const
fn_prime_out.shape

a_map = a.mean(axis=0)
obs_map = obs.mean(axis=0)


In [ ]:
for c in combinations:
    